In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

# Load and prepare data
def load_data():
    # For Kaggle competition, load data from CSV files
    train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
    test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
    
    # Extract labels and features
    y_train = train['label'].values
    X_train = train.drop('label', axis=1).values
    X_test = test.values
    
    # Reshape and normalize
    X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
    X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
    
    # One-hot encode labels
    y_train = to_categorical(y_train, num_classes=10)
    
    return X_train, y_train, X_test

# Define CNN model with batch normalization
def create_model():
    model = Sequential()
    
    # First convolutional block
    model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.25))
    
    # Second convolutional block
    model.add(Conv2D(64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.25))
    
    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Learning rate scheduler
def lr_schedule(epoch):
    initial_lr = 0.001
    if epoch >= 10:
        return initial_lr * 0.5
    if epoch >= 15:
        return initial_lr * 0.1
    return initial_lr

# Create and train ensemble
def train_ensemble(X_train, y_train, n_models=7):
    models = []
    
    # Data augmentation
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1
    )
    
    # Create validation set
    X_train_full, X_val, y_train_full, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=42
    )
    
    # Train each model
    for i in range(n_models):
        print(f"Training model {i+1}/{n_models}")
        
        # Create and compile model
        model = create_model()
        
        # Train with data augmentation and learning rate scheduler
        lr_scheduler = LearningRateScheduler(lr_schedule)
        
        # Shuffle data differently for each model
        X_train_shuffle, y_train_shuffle = shuffle_data(X_train_full, y_train_full, seed=i)
        
        # Fit model
        model.fit(
            datagen.flow(X_train_shuffle, y_train_shuffle, batch_size=64),
            epochs=20,
            validation_data=(X_val, y_val),
            callbacks=[lr_scheduler]
        )
        
        models.append(model)
    
    return models

# Helper function to shuffle data
def shuffle_data(X, y, seed=None):
    indices = np.arange(X.shape[0])
    np.random.seed(seed)
    np.random.shuffle(indices)
    return X[indices], y[indices]

# Generate ensemble predictions
def ensemble_predictions(models, X_test):
    predictions = np.zeros((X_test.shape[0], 10))
    
    for model in models:
        pred = model.predict(X_test)
        predictions += pred
    
    predictions /= len(models)
    return np.argmax(predictions, axis=1)

# Main execution
def main():
    # Load data
    X_train, y_train, X_test = load_data()
    
    # Train ensemble
    models = train_ensemble(X_train, y_train, n_models=7)
    
    # Generate predictions
    predictions = ensemble_predictions(models, X_test)
    
    # Save predictions to CSV for Kaggle submission
    submission = pd.DataFrame({
        'ImageId': range(1, len(predictions) + 1),
        'Label': predictions
    })
    submission.to_csv('submission.csv', index=False)
    print("Submission file created!")

if __name__ == "__main__":
    main()
